In [2]:
import os
import glob
import pandas as pd
import pyndl.io

In [7]:
basepath = '/Users/gracer/Desktop/reversal_learning_logfiles'

# Basic structure
| State | NextState | Reward | Action |  

## Where:
* State = which logo is the reward. There are two possible states
** S1 = Sweet logo is correct
** S2 = Unsweet logo is correct
* Next_state = which state is next, this is dependent on the number of correct
* Reward = Was the action rewarded
** Yes = 1
** No = 2
** Miss = 0
* Action = this is the logo chosen, the participant can choose either the sweetLogo or unsweetLogo or miss

In [8]:
os.path.exists(basepath)

True

In [26]:
data_dict = {'run01':{},'run02':{}}

In [62]:
def find_action_reward(item):
    df = pd.read_csv(item, sep = '\t', header = None)
    X = df[df[1].str.match('Level injecting via pump at address')] 
    Y = X[1].str.split(' ',  expand=True)
    Z = Y[[6,15]]
    Z = Z.rename(columns={6: 'reward', 15: 'Action'})
    return(Z)

Key Press Missed! 

In [91]:
def find_misses(item):
    df = pd.read_csv(item, sep = '\t', header = None)
    X = df[df[1].str.match('Level Key Press Missed')] 
    X['reward'] = 0
    X['Action']= 'miss'
    X = X[['reward','Action']]
    return(X)

In [192]:
def find_correct(item):
    df = pd.read_csv(item, sep = '\t', header = None)
    try:
        X = df[df[1].str.match('Level correct responses')]
        Y = X[1].str.split(' ',  expand=True)
        Z = Y[[3]]
        Z = Z.rename(columns={3: 'corrects'})

        return(Z)
    except KeyError:
        print('no correct responses')
        return('STOP')

In [150]:
def find_flip(item):
    df = pd.read_csv(item, sep = '\t', header = None)
    try:
        X = df[df[1].str.match('Level FLIP')] 
        Y = X[1].str.split(' ',  expand=True)
        Z = Y[[1]]
        Z = Z.rename(columns={1: 'flip'})
        return(Z)
    except KeyError:
        print('no flips')
        return('STOP')
    

In [289]:
data_dict = {'run01':{},'run02':{}}
files = glob.glob(os.path.join(basepath,'*.log'))
files.sort()

for item in files:
    sub = item.split('/')[-1].split('_')[1]
    run = item.split('/')[-1].split('_')[2]
    df1 = find_action_reward(item)
    df2 = find_misses(item)
    df4 = find_flip(item)
    
    
    df_list = [df1, df2,  df4]
    for count, data in enumerate(df_list):
        if isinstance(data, pd.DataFrame):
            print('df%s is good'%count)
        else:
            print('df%s is bad kill'%count)
            data_dict[run][sub] = 'Fail'
            break
    DF = pd.concat(df_list, axis = 0)
    DF = DF.sort_index()
    
    new = []
    states = ['s1','s2']
    next_state = ['s1']

    test = DF['flip'] 
    new2 = [0]*len(test)
    
    for i,x in enumerate(test):
        if x == 'FLIP':
            states = list(reversed(states))
            new2[i-1] = states[0]
        new.append(states[0])
        new2[i] = states[0]
    DF['states'] = new
    DF['next_states'] = new2
    DF = DF.drop('flip', axis=1)
    DF = DF.dropna()
    data_dict[run][sub] = DF
    outpath = os.path.join(basepath, 'parsed','sub-%s_%s.csv'%(sub,run))
    DF.to_csv(outpath)

/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good


/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good


/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

...

df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good


/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
df0 is good
df1 is good
df2 is good
no flips
df0 is good
df1 is good
df2 is bad kill


/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['reward'] = 0
/var/folders/02/zfp2v0gd46j44wl8qx8jxjtr0000gn/T/ipykernel_1820/1020326948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Action']= 'miss'


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [286]:
data_dict['run01']['005']

,reward,Action,states,next_states
14,1,CO.jpg,s1,s1
35,1,CO.jpg,s1,s1
56,1,CO.jpg,s1,s2
79,2,CO.jpg,s2,s2
98,2,CO.jpg,s2,s2
118,1,UCO.jpg,s2,s2
138,1,UCO.jpg,s2,s2
158,1,UCO.jpg,s2,s2
180,1,UCO.jpg,s2,s1
203,2,UCO.jpg,s1,s1


In [290]:
import pickle

In [291]:
outpath = os.path.join(basepath, 'parsed','RL.pickle')
with open(outpath, 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)